In [1]:
import matplotlib.pyplot as plt
import numpy as np
import progressbar

In [2]:
# results in this format from the output of the multi-threaded c++ code.
resultsFinal = np.asarray([
0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 0,
3e-08, 0, 0, 0, 0, 0,
6e-08, 0, 0, 0, 0, 0,
2.5e-07, 0, 0, 0, 0, 0,
7.1e-07, 0, 0, 0, 0, 0,
1.65e-06, 0, 0, 0, 0, 0,
4.9e-06, 0, 0, 0, 0, 0,
1.178e-05, 0, 0, 0, 0, 0,
2.797e-05, 0, 0, 0, 0, 0,
6.254e-05, 0, 0, 0, 0, 0,
0.00013422, 2e-08, 0, 0, 0, 0,
0.00027849, 7e-08, 0, 0, 0, 0,
0.00054056, 1.5e-07, 0, 0, 0, 0,  
0.00101865, 8.4e-07, 0, 0, 0, 0,
0.00185053, 3.71e-06, 0, 0, 0, 0,
0.00323235, 1.331e-05, 0, 0, 0, 0,
0.0054424, 3.902e-05, 0, 0, 0, 0,
0.00889944, 0.00011233, 1e-08, 0, 0, 0,
0.0140104, 0.00029576, 8e-08, 0, 0, 0,
0.0214406, 0.00073342, 1.19e-06, 0, 0, 0,
0.0318141, 0.00168044, 6.31e-06, 2e-08, 0, 0,
0.0459994, 0.00364552, 3.162e-05, 2e-08, 0, 0,
0.0646293, 0.00743669, 0.0001314, 4.6e-07, 0, 0,
0.0885937, 0.0142396, 0.00049523, 4.14e-06, 2e-08, 0,
0.118352, 0.0257663, 0.00162757, 3.233e-05, 3.6e-07, 0, 
0.154433, 0.0440788, 0.00472106, 0.00020428, 3.66e-06, 1e-08, 
0.196837, 0.0715445, 0.0122646, 0.00105628, 4.37e-05, 9.2e-07, 
0.245502, 0.110379, 0.0283845, 0.00443653, 0.00041492, 2.269e-05, 
0.299761, 0.162142, 0.0591246, 0.0153717, 0.00283166, 0.00037416, 
0.358843, 0.227416, 0.111237, 0.0440746, 0.0141348, 0.00365242, 
0.421386, 0.305263, 0.189755, 0.105458, 0.0524408,  0.0232268,
0.485932, 0.392829, 0.295232, 0.212378, 0.146406, 0.0962258, 
0.550785, 0.486158, 0.421551, 0.364836, 0.314247, 0.268755,])
resultsFinal = np.reshape(resultsFinal, (40,6))

In [15]:
# honest network delay over next n blocks.
def vectorDelayHonest(ps, es, init_endorsers, delay_priority, delay_endorse):
    return (60 * len(ps)
           + delay_priority * sum(ps) 
           + sum([delay_endorse * max(init_endorsers - e, 0) for e in es]))

# attacking network delay over next n blocks.
def vectorDelayAttacker(ps, es, init_endorsers, delay_priority, delay_endorse):
    return (60 * len(ps) 
           + delay_priority * sum(ps) 
           + sum([delay_endorse * max(init_endorsers - e, 0) for e in es[1:]]))

# efficient sample generation
def getAH(alpha):
    x = np.random.geometric(1-alpha)
    if x == 1:
        h = 0
        a = np.random.geometric(alpha)
    else:
        a = 0
        h = x - 1
    return [a, h]

def rewardBlock(p, e):
    if p == 0:
        return e * 1.25
    return e * 0.1875

def rewardEndorsement(p):
    if p == 0:
        return 1.25
    return 0.8333333

def calcHonestSingle(p, e):
    if p == 0:
        return rewardBlock(0, 32) + e * rewardEndorsement(0)
    return e * rewardEndorsement(0)

def calcAttackSingle(p, e):
    return rewardBlock(p, e) + e * rewardEndorsement(p)

def vectorRewardHonest(ps, es):
    totalReward = 0
    for i in range(len(ps)):
        totalReward += calcHonestSingle(ps[i], es[i])
    return totalReward

def vectorRewardAttack(ps, es):
    totalReward = calcAttackSingle(ps[0], 32)
    for i in range(1,len(ps)):
        totalReward += calcAttackSingle(ps[i], es[i])
    return totalReward

def calcCosts(ps, es):
    return vectorRewardHonest(ps, es) - vectorRewardAttack(ps, es)

In [35]:
def getProbReorg(alpha, length, init_endorsers, delay_priority, delay_endorse, sample_size = int(1e3)):
    feasible_count = 0
    for _ in range(sample_size):
        aVals = []
        hVals = []
        for i in range(length):
            a, h = getAH(alpha)
            aVals.append(a)
            hVals.append(h)
        eVals = np.random.binomial(32, alpha, size = length)
        honest_delay = vectorDelayHonest(hVals, 32 - eVals, init_endorsers, delay_priority, delay_endorse)
        selfish_delay = vectorDelayAttacker(aVals, eVals, init_endorsers, delay_priority, delay_endorse)
        if selfish_delay <= honest_delay:
            feasible_count += 1
    return feasible_count / sample_size

def getProbSelfish(alpha, length, init_endorsers, delay_priority, delay_endorse, sample_size = int(1e3)):
    feasible_count = 0
    for _ in range(sample_size):
        aVals = []
        hVals = []
        for i in range(length):
            a, h = getAH(alpha)
            aVals.append(a)
            hVals.append(h)
        eVals = np.random.binomial(32, alpha, size = length)
        honest_delay = vectorDelayHonest(hVals, 32 - eVals, init_endorsers, delay_priority, delay_endorse)
        selfish_delay = vectorDelayAttacker(aVals, eVals, init_endorsers, delay_priority, delay_endorse)
        if (selfish_delay <= honest_delay) and (calcCosts(aVals, eVals) < 0):
            feasible_count += 1
    return feasible_count / sample_size

In [6]:
length_20_probs = resultsFinal[:,2]
length_20_probs_nonzero = length_20_probs[24:]
length_20_probs_nonzero

array([1.00000e-08, 8.00000e-08, 1.19000e-06, 6.31000e-06, 3.16200e-05,
       1.31400e-04, 4.95230e-04, 1.62757e-03, 4.72106e-03, 1.22646e-02,
       2.83845e-02, 5.91246e-02, 1.11237e-01, 1.89755e-01, 2.95232e-01,
       4.21551e-01])

In [7]:
alphas_total= np.arange(0.34, 0.50, 0.01)
for i in range(16):
    print(alphas_total[i], length_20_probs_nonzero[i])

0.34 1e-08
0.35000000000000003 8e-08
0.36000000000000004 1.19e-06
0.37000000000000005 6.31e-06
0.38000000000000006 3.162e-05
0.39000000000000007 0.0001314
0.4000000000000001 0.00049523
0.4100000000000001 0.00162757
0.4200000000000001 0.00472106
0.4300000000000001 0.0122646
0.4400000000000001 0.0283845
0.4500000000000001 0.0591246
0.46000000000000013 0.111237
0.47000000000000014 0.189755
0.48000000000000015 0.295232
0.49000000000000016 0.421551


In [8]:
alphas = [0.41, 0.45, 0.49]
tezos_probs = [length_20_probs_nonzero[7], length_20_probs_nonzero[11], length_20_probs_nonzero[15]]
for i in range(3):
    print(alphas[i], tezos_probs[i])
weights = 1 / np.asarray(tezos_probs)
weights

0.41 0.00162757
0.45 0.0591246
0.49 0.421551


array([614.41289776,  16.91343366,   2.37219221])

In [9]:
init_endorsers_range = list(range(33))
delay_priority_range = list(range(20, 61))
delay_endorse_range = list(range(4, 13))

In [10]:
results_grid = np.zeros(shape=(
    len(init_endorsers_range),
    len(delay_priority_range),
    len(delay_endorse_range)))

In [11]:
alphas, weights

([0.41, 0.45, 0.49], array([614.41289776,  16.91343366,   2.37219221]))

In [69]:
prob1 = getProbReorg(
                alpha=0.45, 
                length=20, 
                init_endorsers=24,
                delay_priority=40,
                delay_endorse=8,
                sample_size=int(1e5))
prob2 = getProbSelfish(
                alpha=0.45, 
                length=3, 
                init_endorsers=24,
                delay_priority=40,
                delay_endorse=8,
                sample_size=int(1e5))
print(prob1, prob2)

0.05974 0.07243


In [65]:
bar = progressbar.ProgressBar()
for i in bar(range(len(init_endorsers_range))):
    for j in range(len(delay_priority_range)):
        for k in range(len(delay_endorse_range)):
            prob1 = getProbReorg(
                alpha=0.45, 
                length=20, 
                init_endorsers=init_endorsers_range[i],
                delay_priority=delay_priority_range[j],
                delay_endorse=delay_endorse_range[k],
                sample_size=int(1e3))
            prob2 = getProbSelfish(
                alpha=0.45, 
                length=3, 
                init_endorsers=init_endorsers_range[i],
                delay_priority=delay_priority_range[j],
                delay_endorse=delay_endorse_range[k],
                sample_size=int(1e3))
#             prob2 = getProbReorg(
#                 alphas[1], 
#                 length=20, 
#                 init_endorsers=init_endorsers_range[i],
#                 delay_priority=delay_priority_range[j],
#                 delay_endorse=delay_endorse_range[k],
#                 sample_size=int(1e3))
#             prob3 = getProbReorg(
#                 alphas[2], 
#                 length=20, 
#                 init_endorsers=init_endorsers_range[i],
#                 delay_priority=delay_priority_range[j],
#                 delay_endorse=delay_endorse_range[k],
#                 sample_size=int(1e3))
#             val = weights[0] * prob1 + weights[1] * prob2 + weights[2] * prob3
            results_grid[i, j, k] = prob1 + prob2

100% (33 of 33) |########################| Elapsed Time: 0:23:02 Time:  0:23:02


In [66]:
list(results_grid)

[array([[0.328, 0.318, 0.315, 0.305, 0.313, 0.341, 0.336, 0.315, 0.315],
        [0.344, 0.317, 0.329, 0.341, 0.314, 0.346, 0.308, 0.304, 0.278],
        [0.327, 0.33 , 0.341, 0.324, 0.269, 0.313, 0.339, 0.343, 0.326],
        [0.328, 0.342, 0.315, 0.285, 0.315, 0.339, 0.352, 0.304, 0.34 ],
        [0.314, 0.312, 0.285, 0.347, 0.295, 0.296, 0.323, 0.31 , 0.33 ],
        [0.305, 0.301, 0.309, 0.337, 0.317, 0.321, 0.308, 0.336, 0.335],
        [0.304, 0.31 , 0.306, 0.349, 0.334, 0.313, 0.321, 0.323, 0.303],
        [0.324, 0.307, 0.317, 0.302, 0.318, 0.287, 0.321, 0.319, 0.293],
        [0.352, 0.343, 0.32 , 0.321, 0.322, 0.32 , 0.316, 0.326, 0.297],
        [0.318, 0.325, 0.313, 0.308, 0.349, 0.321, 0.333, 0.315, 0.323],
        [0.329, 0.323, 0.339, 0.318, 0.293, 0.3  , 0.316, 0.318, 0.3  ],
        [0.351, 0.312, 0.326, 0.323, 0.305, 0.313, 0.321, 0.326, 0.328],
        [0.317, 0.299, 0.311, 0.315, 0.329, 0.321, 0.331, 0.333, 0.324],
        [0.323, 0.297, 0.334, 0.31 , 0.333, 0.308, 

In [67]:
np.savetxt('grid45_cost.txt', results_grid.flatten())